In [4]:
import sys
import os

# Add the build directory to Python path
build_dir = os.path.abspath('/Users/kisel/uni/apal/Algorithm-Selector/Implementation/lib')
sys.path.insert(0, build_dir)

# Add the data directory to Python path
data_dir = os.path.abspath('/Users/kisel/uni/apal/Algorithm-Selector/Implementation/data')
sys.path.insert(0, data_dir)

import pandas as pd
import numpy as np
from tqdm import tqdm

import sorters
import probes
import time

import list_generators as lg
import runtime_features as rf

In [7]:
data = pd.read_feather("/Users/kisel/uni/apal/Algorithm-Selector/Implementation/data/varied_dataset.feather")
data = data.head(10000)
data

,list_type,list
0,random,"[7.0, 8.0, 1.0, 0.0, 3.0, 2.0, 5.0, 6.0, 9.0, ..."
1,reversed,"[9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, ..."
2,disorder_0.05,"[0.0, 1.0, 2.0, 3.0, 6.0, 5.0, 4.0, 7.0, 8.0, ..."
3,disorder_0.10,"[0.0, 1.0, 2.0, 7.0, 4.0, 5.0, 6.0, 3.0, 8.0, ..."
4,disorder_0.20,"[0.0, 1.0, 8.0, 3.0, 4.0, 7.0, 6.0, 5.0, 2.0, ..."
...,...,...
79,normal,"[36.37213293458489, 45.99389482462006, 44.1843..."
80,exponential,"[0.4549158106086768, 1.0463992088469318, 0.515..."
81,zipf,"[1.0, 3.0, 1.0, 1.0, 5.0, 4.0, 1.0, 1.0, 1.0, ..."
82,clustered,"[86.10939884639929, 85.42753516586251, 85.1785..."


In [3]:
data['freq'] = data['data'].apply(rf.find_frequency)
data

,data,freq
0,[],{}
1,"[0, 1, 2, 3, 4, 5, 6, 13, 8, 9, 44, 11, 12, 7,...","{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 13:..."
2,"[47, 46, 45, 10, 43, 42, 41, 40, 39, 38, 37, 3...","{47: 1, 46: 1, 45: 1, 10: 1, 43: 1, 42: 1, 41:..."
3,"[0, 1, 2, 3, 4, 5, 6, 7, 33, 9, 10, 11, 12, 13...","{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: ..."
4,"[47, 46, 45, 44, 43, 42, 24, 21, 39, 38, 37, 3...","{47: 1, 46: 1, 45: 1, 44: 1, 43: 1, 42: 1, 24:..."
...,...,...
9995,"[0, 1, 301, 2, 3, 3, 4, 4, 4, 5, 5, 6, 1196, 6...","{0: 1, 1: 1, 301: 1, 2: 2, 3: 2, 4: 3, 5: 2, 6..."
9996,"[184, 103, 1784, 1783, 432, 1783, 1783, 1782, ...","{184: 1, 103: 1, 1784: 3, 1783: 4, 432: 2, 178..."
9997,"[0, 601, 237, 2, 3, 3, 4, 665, 4, 606, 5, 902,...","{0: 1, 601: 2, 237: 1, 2: 2, 3: 2, 4: 3, 665: ..."
9998,"[1784, 1187, 1018, 1366, 274, 1783, 1783, 1782...","{1784: 3, 1187: 1, 1018: 1, 1366: 1, 274: 1, 1..."


In [4]:
data['avg_dup_position'] = data['data'].apply(rf.find_avg_dup_pos)
data['avg_dup_distinct'] = data['data'].apply(rf.find_avg_dup_distinct)

data

,data,freq,avg_dup_position,avg_dup_distinct
0,[],{},0.000000,0.000000
1,"[0, 1, 2, 3, 4, 5, 6, 13, 8, 9, 44, 11, 12, 7,...","{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 13:...",0.080000,0.041667
2,"[47, 46, 45, 10, 43, 42, 41, 40, 39, 38, 37, 3...","{47: 1, 46: 1, 45: 1, 10: 1, 43: 1, 42: 1, 41:...",0.080000,0.041667
3,"[0, 1, 2, 3, 4, 5, 6, 7, 33, 9, 10, 11, 12, 13...","{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: ...",0.080000,0.041667
4,"[47, 46, 45, 44, 43, 42, 24, 21, 39, 38, 37, 3...","{47: 1, 46: 1, 45: 1, 44: 1, 43: 1, 42: 1, 24:...",0.080000,0.041667
...,...,...,...,...
9995,"[0, 1, 301, 2, 3, 3, 4, 4, 4, 5, 5, 6, 1196, 6...","{0: 1, 1: 1, 301: 1, 2: 2, 3: 2, 4: 3, 5: 2, 6...",0.738824,0.428571
9996,"[184, 103, 1784, 1783, 432, 1783, 1783, 1782, ...","{184: 1, 103: 1, 1784: 3, 1783: 4, 432: 2, 178...",0.738824,0.428571
9997,"[0, 601, 237, 2, 3, 3, 4, 665, 4, 606, 5, 902,...","{0: 1, 601: 2, 237: 1, 2: 2, 3: 2, 4: 3, 665: ...",0.738824,0.428571
9998,"[1784, 1187, 1018, 1366, 274, 1783, 1783, 1782...","{1784: 3, 1187: 1, 1018: 1, 1366: 1, 274: 1, 1...",0.738824,0.428571


In [5]:
data['entropy'] = data['data'].apply(rf.find_entropy)
data

,data,freq,avg_dup_position,avg_dup_distinct,entropy
0,[],{},0.000000,0.000000,0.000000
1,"[0, 1, 2, 3, 4, 5, 6, 13, 8, 9, 44, 11, 12, 7,...","{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 13:...",0.080000,0.041667,5.563856
2,"[47, 46, 45, 10, 43, 42, 41, 40, 39, 38, 37, 3...","{47: 1, 46: 1, 45: 1, 10: 1, 43: 1, 42: 1, 41:...",0.080000,0.041667,5.563856
3,"[0, 1, 2, 3, 4, 5, 6, 7, 33, 9, 10, 11, 12, 13...","{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: ...",0.080000,0.041667,5.563856
4,"[47, 46, 45, 44, 43, 42, 24, 21, 39, 38, 37, 3...","{47: 1, 46: 1, 45: 1, 44: 1, 43: 1, 42: 1, 24:...",0.080000,0.041667,5.563856
...,...,...,...,...,...
9995,"[0, 1, 301, 2, 3, 3, 4, 4, 4, 5, 5, 6, 1196, 6...","{0: 1, 1: 1, 301: 1, 2: 2, 3: 2, 4: 3, 5: 2, 6...",0.738824,0.428571,10.666839
9996,"[184, 103, 1784, 1783, 432, 1783, 1783, 1782, ...","{184: 1, 103: 1, 1784: 3, 1783: 4, 432: 2, 178...",0.738824,0.428571,10.666839
9997,"[0, 601, 237, 2, 3, 3, 4, 665, 4, 606, 5, 902,...","{0: 1, 601: 2, 237: 1, 2: 2, 3: 2, 4: 3, 665: ...",0.738824,0.428571,10.666839
9998,"[1784, 1187, 1018, 1366, 274, 1783, 1783, 1782...","{1784: 3, 1187: 1, 1018: 1, 1366: 1, 274: 1, 1...",0.738824,0.428571,10.666839


In [6]:
data['dis'] = data['data'].apply(rf.find_dis)
data['mono'] = data['data'].apply(rf.find_mono)
data['runs'] = data['data'].apply(rf.find_runs)

data

,data,freq,avg_dup_position,avg_dup_distinct,entropy,dis,mono,runs
0,[],{},0.000000,0.000000,0.000000,0,0,0
1,"[0, 1, 2, 3, 4, 5, 6, 13, 8, 9, 44, 11, 12, 7,...","{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 13:...",0.080000,0.041667,5.563856,39,4,4
2,"[47, 46, 45, 10, 43, 42, 41, 40, 39, 38, 37, 3...","{47: 1, 46: 1, 45: 1, 10: 1, 43: 1, 42: 1, 41:...",0.080000,0.041667,5.563856,49,4,43
3,"[0, 1, 2, 3, 4, 5, 6, 7, 33, 9, 10, 11, 12, 13...","{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: ...",0.080000,0.041667,5.563856,35,7,7
4,"[47, 46, 45, 44, 43, 42, 24, 21, 39, 38, 37, 3...","{47: 1, 46: 1, 45: 1, 44: 1, 43: 1, 42: 1, 24:...",0.080000,0.041667,5.563856,49,7,41
...,...,...,...,...,...,...,...,...
9995,"[0, 1, 301, 2, 3, 3, 4, 4, 4, 5, 5, 6, 1196, 6...","{0: 1, 1: 1, 301: 1, 2: 2, 3: 2, 4: 3, 5: 2, 6...",0.738824,0.428571,10.666839,2547,684,712
9996,"[184, 103, 1784, 1783, 432, 1783, 1783, 1782, ...","{184: 1, 103: 1, 1784: 3, 1783: 4, 432: 2, 178...",0.738824,0.428571,10.666839,2549,684,1499
9997,"[0, 601, 237, 2, 3, 3, 4, 665, 4, 606, 5, 902,...","{0: 1, 601: 2, 237: 1, 2: 2, 3: 2, 4: 3, 665: ...",0.738824,0.428571,10.666839,2544,793,825
9998,"[1784, 1187, 1018, 1366, 274, 1783, 1783, 1782...","{1784: 3, 1187: 1, 1018: 1, 1366: 1, 274: 1, 1...",0.738824,0.428571,10.666839,2549,793,1457


In [7]:
data['skewness'] = data['data'].apply(rf.find_skewness)
data['kurtosis'] = data['data'].apply(rf.find_kurtosis)

data

/Users/kisel/uni/apal/Algorithm-Selector/Implementation/lib/runtime_features.py:64: RuntimeWarning: Mean of empty slice.
  m = a.mean()
/Users/kisel/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/kisel/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/kisel/uni/apal/Algorithm-Selector/Implementation/lib/runtime_features.py:72: RuntimeWarning: Mean of empty slice.
  m = a.mean()
/Users/kisel/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/kisel/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


,data,freq,avg_dup_position,avg_dup_distinct,entropy,dis,mono,runs,skewness,kurtosis
0,[],{},0.000000,0.000000,0.000000,0,0,0,NaN,NaN
1,"[0, 1, 2, 3, 4, 5, 6, 13, 8, 9, 44, 11, 12, 7,...","{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 13:...",0.080000,0.041667,5.563856,39,4,4,-0.021716,-1.131363
2,"[47, 46, 45, 10, 43, 42, 41, 40, 39, 38, 37, 3...","{47: 1, 46: 1, 45: 1, 10: 1, 43: 1, 42: 1, 41:...",0.080000,0.041667,5.563856,49,4,43,-0.021716,-1.131363
3,"[0, 1, 2, 3, 4, 5, 6, 7, 33, 9, 10, 11, 12, 13...","{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: ...",0.080000,0.041667,5.563856,35,7,7,-0.021716,-1.131363
4,"[47, 46, 45, 44, 43, 42, 24, 21, 39, 38, 37, 3...","{47: 1, 46: 1, 45: 1, 44: 1, 43: 1, 42: 1, 24:...",0.080000,0.041667,5.563856,49,7,41,-0.021716,-1.131363
...,...,...,...,...,...,...,...,...,...,...
9995,"[0, 1, 301, 2, 3, 3, 4, 4, 4, 5, 5, 6, 1196, 6...","{0: 1, 1: 1, 301: 1, 2: 2, 3: 2, 4: 3, 5: 2, 6...",0.738824,0.428571,10.666839,2547,684,712,-0.014475,-1.198292
9996,"[184, 103, 1784, 1783, 432, 1783, 1783, 1782, ...","{184: 1, 103: 1, 1784: 3, 1783: 4, 432: 2, 178...",0.738824,0.428571,10.666839,2549,684,1499,-0.014475,-1.198292
9997,"[0, 601, 237, 2, 3, 3, 4, 665, 4, 606, 5, 902,...","{0: 1, 601: 2, 237: 1, 2: 2, 3: 2, 4: 3, 665: ...",0.738824,0.428571,10.666839,2544,793,825,-0.014475,-1.198292
9998,"[1784, 1187, 1018, 1366, 274, 1783, 1783, 1782...","{1784: 3, 1187: 1, 1018: 1, 1366: 1, 274: 1, 1...",0.738824,0.428571,10.666839,2549,793,1457,-0.014475,-1.198292


In [8]:
data.drop(['freq'], axis=1, inplace=True)
data

,data,avg_dup_position,avg_dup_distinct,entropy,dis,mono,runs,skewness,kurtosis
0,[],0.000000,0.000000,0.000000,0,0,0,NaN,NaN
1,"[0, 1, 2, 3, 4, 5, 6, 13, 8, 9, 44, 11, 12, 7,...",0.080000,0.041667,5.563856,39,4,4,-0.021716,-1.131363
2,"[47, 46, 45, 10, 43, 42, 41, 40, 39, 38, 37, 3...",0.080000,0.041667,5.563856,49,4,43,-0.021716,-1.131363
3,"[0, 1, 2, 3, 4, 5, 6, 7, 33, 9, 10, 11, 12, 13...",0.080000,0.041667,5.563856,35,7,7,-0.021716,-1.131363
4,"[47, 46, 45, 44, 43, 42, 24, 21, 39, 38, 37, 3...",0.080000,0.041667,5.563856,49,7,41,-0.021716,-1.131363
...,...,...,...,...,...,...,...,...,...
9995,"[0, 1, 301, 2, 3, 3, 4, 4, 4, 5, 5, 6, 1196, 6...",0.738824,0.428571,10.666839,2547,684,712,-0.014475,-1.198292
9996,"[184, 103, 1784, 1783, 432, 1783, 1783, 1782, ...",0.738824,0.428571,10.666839,2549,684,1499,-0.014475,-1.198292
9997,"[0, 601, 237, 2, 3, 3, 4, 665, 4, 606, 5, 902,...",0.738824,0.428571,10.666839,2544,793,825,-0.014475,-1.198292
9998,"[1784, 1187, 1018, 1366, 274, 1783, 1783, 1782...",0.738824,0.428571,10.666839,2549,793,1457,-0.014475,-1.198292


In [9]:
def find_best_sorter(arr):
    sorts = sorters.list_sorters()
    if 'counting_sort' in sorts:
        sorts.remove('counting_sort')
        sorts.remove('spread_sort')
        sorts.remove('ska_sort')
        
    times = {}
    for sorter in sorts:
        benchmark_time = sorters.benchmark_sorter(sorter, arr)
        times[sorter] = benchmark_time
    
    best_sorter = min(times, key=times.get)
    return best_sorter

In [10]:
tqdm.pandas()
data['best_sort'] = data['data'].progress_apply(find_best_sorter)

100%|██████████| 10000/10000 [00:06<00:00, 1455.06it/s]


In [11]:
data['best_sort'].describe()

count        10000
unique           7
top       std_sort
freq          5707
Name: best_sort, dtype: object

In [12]:
data

,data,avg_dup_position,avg_dup_distinct,entropy,dis,mono,runs,skewness,kurtosis,best_sort
0,[],0.000000,0.000000,0.000000,0,0,0,NaN,NaN,spin_sort
1,"[0, 1, 2, 3, 4, 5, 6, 13, 8, 9, 44, 11, 12, 7,...",0.080000,0.041667,5.563856,39,4,4,-0.021716,-1.131363,insertion_sort
2,"[47, 46, 45, 10, 43, 42, 41, 40, 39, 38, 37, 3...",0.080000,0.041667,5.563856,49,4,43,-0.021716,-1.131363,quick_merge_sort
3,"[0, 1, 2, 3, 4, 5, 6, 7, 33, 9, 10, 11, 12, 13...",0.080000,0.041667,5.563856,35,7,7,-0.021716,-1.131363,insertion_sort
4,"[47, 46, 45, 44, 43, 42, 24, 21, 39, 38, 37, 3...",0.080000,0.041667,5.563856,49,7,41,-0.021716,-1.131363,quick_sort
...,...,...,...,...,...,...,...,...,...,...
9995,"[0, 1, 301, 2, 3, 3, 4, 4, 4, 5, 5, 6, 1196, 6...",0.738824,0.428571,10.666839,2547,684,712,-0.014475,-1.198292,std_sort
9996,"[184, 103, 1784, 1783, 432, 1783, 1783, 1782, ...",0.738824,0.428571,10.666839,2549,684,1499,-0.014475,-1.198292,quick_sort
9997,"[0, 601, 237, 2, 3, 3, 4, 665, 4, 606, 5, 902,...",0.738824,0.428571,10.666839,2544,793,825,-0.014475,-1.198292,std_sort
9998,"[1784, 1187, 1018, 1366, 274, 1783, 1783, 1782...",0.738824,0.428571,10.666839,2549,793,1457,-0.014475,-1.198292,spin_sort


In [13]:
data.to_feather("/Users/kisel/uni/apal/Algorithm-Selector/Implementation/data/data_and_features.feather")